## Pruebas seleccion de puntos de muestreo



In [32]:
# If running this notebook in Mac
#%matplotlib osx

In [1]:
# If running this notebook in Ubuntu
%matplotlib tk

In [2]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely

In [3]:
# Número total de muestras es seleccionado asumiendo que 
# cada ciclo de medición abarca 2 años
# que se dispondrán de dos brigadas para la fase de campo 
# que cada brigada visitaría 1 localidad distinta por semana, 
# que cada contrato durará 10 meses por año

total_samples = 2 * 10 * 4 * 2
km_dr = 180 / 40075

In [4]:
locafile = '../Datos/GIS/localidades_bogota/Loca.shp'
loca = gpd.read_file(locafile, encoding='ascii')
loca = loca.to_crs('EPSG:4686')

In [5]:
clc = '../Datos/GIS/cobertura_tierra_clc_2018_Cundinamarca_Bogota_GCS_MAGNA/cobertura_tierra_clc_2018_Cundinamarca_Bogota_GCS_MAGNA.shp'
cob = gpd.read_file(clc, encoding='ascii')
#cob = cob.to_crs('EPSG:4326')

In [6]:
simpl = {
	'Zonas verdes artificializadas':[
		'1.4.1. Zonas verdes urbanas',
		'1.4.1.2. Parques cementerio',
		'1.4.1.3. Jardines botánicos',
		'1.4.2. Instalaciones recreativas'
	],
	'Territorios artificializados':[
		'1.1.1. Tejido urbano continuo',
		'1.1.2. Tejido urbano discontinuo',
		'1.2.1. Zonas industriales o comerciales',
		'1.2.1.1. Zonas industriales',
		'1.2.2. Red vial, ferroviaria y terrenos asociados',
		'1.2.2.1. Red vial y territorios asociados',
		'1.2.4. Aeropuertos',
		'1.2.5. Obras hidráulicas',
		'1.3.1. Zonas de extracción minera',
		'1.3.2. Zona de disposición de residuos'
	],
	'Territorios agrícolas':[
		'2.1.1. Otros cultivos transitorios',
		'2.1.2.1. Arroz',
		'2.1.3. Oleaginosas y leguminosas',
		'2.1.4. Hortalizas',
		'2.1.5. Tubérculos',
		'2.1.5.1. Papa',
		'2.2.1. Cultivos permanentes herbáceos',
		'2.2.1.2. Caña',
		'2.2.2.1. Otros cultivos permanentes arbustivos',
		'2.2.2.2. Café',
		'2.2.3. Cultivos permanentes arbóreos',
		'2.2.3.2. Palma de aceite',
		'2.2.3.4. Mango',
		'2.2.4. Cultivos agroforestales',
		'2.2.5. Cultivos confinados',
		'2.3.1. Pastos limpios',
		'2.3.2. Pastos arbolados',
		'2.3.3. Pastos enmalezados',
		'2.4.1. Mosaico de cultivos',
		'2.4.2. Mosaico de pastos y cultivos',
		'2.4.3. Mosaico de cultivos, pastos y espacios naturales',
		'2.4.4. Mosaico de pastos con espacios naturales',
		'2.4.5. Mosaico de cultivos con espacios naturales',
	],
	'Bosques':[
		'3.1.1.1.1. Bosque denso alto de tierra firme',
		'3.1.1.2.1. Bosque denso bajo de tierra firme',
		'3.1.2.1.1. Bosque abierto alto de tierra firme',
		'3.1.2.2.1. Bosque abierto bajo de tierra firme',
		'3.1.2.2.2. Bosque abierto bajo inundable',
		'3.1.3.1. Bosque fragmentado con pastos y cultivos',
		'3.1.3.2. Bosque fragmentado con vegetación secundaria',
		'3.1.4. Bosque de galería y ripario',
		'3.1.5. Plantación forestal',
	],
	'Herbazales':[
		'3.2.1.1.1.1.  Herbazal denso de tierra firme no arbolado',
		'3.2.1.1.1.2.  Herbazal denso de tierra firme arbolado',
		'3.2.1.1.1.3.  Herbazal denso de tierra firme con arbustos',
		'3.2.1.1.2.1. Herbazal denso inundable no arbolado',
		'3.2.1.2.2. Herbazal abierto rocoso'
	],
	'Arbustales':[
		'3.2.2.1. Arbustal denso',
		'3.2.2.2. Arbustal abierto',
		'3.2.3. Vegetación secundaria o en transición',
		'3.2.3.1. Vegetación secundaria alta',
		'3.2.3.2. Vegetación secundaria baja',
	],
	'Áreas abiertas':[
		'3.3.1. Zonas arenosas naturales',
		'3.3.1.2. Arenales',
		'3.3.2. Afloramientos rocosos',
		'3.3.3. Tierras desnudas y degradadas',
	],
	'Áreas humedas': [
		'4.1.1. Zonas pantanosas',
		'4.1.2. Turberas',
		'4.1.3. Vegetación acuática sobre cuerpos de agua',
	],
	'Superficies de agua': [
		'5.1.1. Ríos',
		'5.1.2. Lagunas, lagos y ciénagas naturales',
		'5.1.3. Canales',
		'5.1.4. Cuerpos de agua artificiales',
		'5.1.4.1. Embalses'
	]
}

In [7]:
cob['leyenda_simple'] = None

for k in simpl:
	for l in simpl[k]:
		cob.loc[cob.leyenda == l, 'leyenda_simple'] = k

In [8]:
# Correcting some ill-formed polygons
# Error only displays in Ubuntu GeoPandas 0.14.2
cob['geometry'] = cob.geometry.buffer(0)

In [9]:
cob = cob.clip(
	loca.dissolve()
)

In [12]:
ls ../Datos/GIS/CLC_Bogota_leyenda_agregada/

In [10]:
cob.plot(column='leyenda_simple', legend=True)

<Axes: >

In [47]:
cob['area_km'] = cob.area / km_dr ** 2

areaxcob = cob[['area_km', 'leyenda_simple']
	].groupby('leyenda_simple').sum()

areaxcob['area_km_sqrt'] = areaxcob.area_km ** 0.5
areaxcob['area_km_cube'] = areaxcob.area_km ** 0.333333
areaxcob['area_km_cuadr'] = areaxcob.area_km ** 0.25
areaxcob['area_km_ln'] = areaxcob.area_km.apply(np.log)

/tmp/ipykernel_34564/371082999.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cob['area_km'] = cob.area / km_dr ** 2


In [48]:
areaxcob = areaxcob.drop([
	'Áreas abiertas', 
	'Áreas humedas', 
	'Territorios artificializados', 
	'Superficies de agua'
])
areaxcob

,area_km,area_km_sqrt,area_km_cube,area_km_cuadr,area_km_ln
leyenda_simple,,,,,
Arbustales,658.993314,25.670865,8.702140,5.066642,6.490713
Bosques,330.801835,18.187959,6.916002,4.264734,5.801520
Herbazales,2503.289426,50.032883,13.578003,7.073393,7.825361
Territorios agrícolas,1596.695840,39.958677,11.687985,6.321288,7.375692
Zonas verdes artificializadas,113.172515,10.638257,4.837040,3.261634,4.728913


In [106]:
area_weights = areaxcob.area_km_ln / areaxcob.area_km_ln.sum() 
(area_weights * total_samples).round()

leyenda_simple
Arbustales                       32.0
Bosques                          29.0
Herbazales                       39.0
Territorios agrícolas            37.0
Zonas verdes artificializadas    23.0
Name: area_km_ln, dtype: float64

In [107]:
area_weights = areaxcob.area_km_cuadr / areaxcob.area_km_cuadr.sum()
(area_weights * total_samples).round()

leyenda_simple
Arbustales                       31.0
Bosques                          26.0
Herbazales                       44.0
Territorios agrícolas            39.0
Zonas verdes artificializadas    20.0
Name: area_km_cuadr, dtype: float64

In [103]:
area_weights =  areaxcob.area_km_sqrt / areaxcob.area_km_sqrt.sum()
(area_weights * total_samples).round()

leyenda_simple
Arbustales                       28.0
Bosques                          20.0
Herbazales                       55.0
Territorios agrícolas            44.0
Zonas verdes artificializadas    12.0
Name: area_km_sqrt, dtype: float64

In [105]:
area_weights = areaxcob.area_km / areaxcob.area_km.sum()
(total_samples * area_weights).round()

leyenda_simple
Arbustales                       20.0
Bosques                          10.0
Herbazales                       77.0
Territorios agrícolas            49.0
Zonas verdes artificializadas     3.0
Name: area_km, dtype: float64

In [49]:
area_weights = areaxcob.area_km_cube / areaxcob.area_km_cube.sum()
(area_weights * total_samples).round()

leyenda_simple
Arbustales                       30.0
Bosques                          24.0
Herbazales                       48.0
Territorios agrícolas            41.0
Zonas verdes artificializadas    17.0
Name: area_km_cube, dtype: float64

In [50]:
area_weights.name = 'Weight'
area_weights

leyenda_simple
Arbustales                       0.190331
Bosques                          0.151265
Herbazales                       0.296974
Territorios agrícolas            0.255636
Zonas verdes artificializadas    0.105794
Name: Weight, dtype: float64

In [51]:
xmin= cob.bounds['minx'].min()
xmax= cob.bounds['maxx'].max()
ymin= cob.bounds['miny'].min()
ymax= cob.bounds['maxy'].max()

In [52]:
cell_size = km_dr * 2
rng = np.random.default_rng()
offset_x = rng.uniform() * cell_size
offset_y = rng.uniform() * cell_size

# projection of the grid
mycrs = 4686
# create the cells in a loop
grid_cells = []
for x0 in np.arange(xmin-offset_x, xmax, cell_size ):
	for y0 in np.arange(ymin-offset_y, ymax, cell_size):
		# bounds
		x1 = x0+cell_size
		y1 = y0+cell_size
		grid_cells.append(shapely.geometry.box(x0, y0, x1, y1)  )
cell = gpd.GeoDataFrame(grid_cells, columns=['geometry'], 
		crs=mycrs)

In [53]:
# Plot the whole sampling frame

ax = cob[cob.leyenda_simple.notna()
	].plot(column='leyenda_simple',
		legend=True,
		cmap='rainbow'
	)
cell.plot(ax=ax, facecolor="none", edgecolor='grey')
#ax.axis("off")

<Axes: >

In [54]:
centers = gpd.GeoDataFrame(cell.geometry.centroid, geometry=0)

/tmp/ipykernel_34564/1666895978.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centers = gpd.GeoDataFrame(cell.geometry.centroid, geometry=0)


In [55]:
cobxcentroids = gpd.overlay(
	centers,
	cob[cob.leyenda_simple.notna()], 
)

In [56]:
cobxcentroids.groupby('leyenda_simple').size()

leyenda_simple
Arbustales                       162
Bosques                           95
Herbazales                       635
Superficies de agua                3
Territorios agrícolas            379
Territorios artificializados     336
Zonas verdes artificializadas     30
Áreas humedas                      8
dtype: int64

In [57]:
area_weights

leyenda_simple
Arbustales                       0.190331
Bosques                          0.151265
Herbazales                       0.296974
Territorios agrícolas            0.255636
Zonas verdes artificializadas    0.105794
Name: Weight, dtype: float64

In [58]:
sampl_cob = cobxcentroids.groupby('leyenda_simple'
	).size()

sampl_cob.name = 'Samples'
samples = pd.concat([
	sampl_cob, 
	total_samples * area_weights / sampl_cob
	], axis=1).dropna()
samples = samples.rename(columns={0: 'sampl_prob'})
samples

,Samples,sampl_prob
leyenda_simple,,
Arbustales,162,0.187981
Bosques,95,0.254762
Herbazales,635,0.074828
Territorios agrícolas,379,0.107920
Zonas verdes artificializadas,30,0.564236


In [59]:
samples.sampl_prob * samples.Samples

leyenda_simple
Arbustales                       30.452904
Bosques                          24.202363
Herbazales                       47.515854
Territorios agrícolas            40.901790
Zonas verdes artificializadas    16.927089
dtype: float64

In [60]:
# Theoretical number of effective samples
samples.sampl_prob / samples.Samples  

leyenda_simple
Arbustales                       0.001160
Bosques                          0.002682
Herbazales                       0.000118
Territorios agrícolas            0.000285
Zonas verdes artificializadas    0.018808
dtype: float64

## Seleccion aleatoria de psus

In [61]:
cobxcentroids['prob'] = rng.uniform(0, 1, cobxcentroids.shape[0])
cobxcentroids['accepted'] = 0

In [62]:
for i in samples.index:
	thres = samples.loc[i, 'sampl_prob']
	cobxcentroids.loc[
		(cobxcentroids.leyenda_simple == i) & 
		(cobxcentroids.prob < thres), 'accepted'
		] = 1

In [63]:
# Actual sample aggregation by strata
cobxcentroids[cobxcentroids.accepted == 1].groupby('leyenda_simple').size()

leyenda_simple
Arbustales                       30
Bosques                          28
Herbazales                       47
Territorios agrícolas            37
Zonas verdes artificializadas    18
dtype: int64

In [64]:
# Plot sampling frame and strata
ax = cob[cob.leyenda_simple.notna()
	].plot(column='leyenda_simple',
		legend=True,
		cmap='rainbow'
	)
cell.plot(
	ax=ax, facecolor="none", edgecolor='grey'
)
#ax.axis("off")

<Axes: >

In [65]:
# Plot all possible sampling sites
ax = cob[cob.leyenda_simple.notna()
	].plot(column='leyenda_simple',
		legend=True,
		cmap='rainbow'
	)
cobxcentroids.plot(ax=ax, color='black')

#ax.axis("off")

<Axes: >

In [67]:
fig, ax = plt.subplots(figsize=(12, 12))

# Set map limits
xlim = ([-74.5, -73.8])
ylim = ([3.7, 4.85])
ax.set_xlim(xlim)
ax.set_ylim(ylim)

f0 = cob[
	cob.leyenda_simple.notna()
].plot(
	ax=ax,
	column='leyenda_simple',
	legend=True,
	legend_kwds={'loc': 'lower right'},
	cmap='rainbow'
)

f1 = cobxcentroids[cobxcentroids.accepted == 1].plot(ax=ax, color='black', markersize=5)

plt.xlabel('Longitud', labelpad=15)
plt.ylabel('Latitud', labelpad=15)
#plt.title('Especie amenazadas y areas protegidas', pad=25)
#plt.show()
plt.savefig('sitios_muestreo.png', dpi=300, bbox_inches='tight')

## Seleccion sistematica de psus

In [27]:
samples

,Samples,Weight
Cobertura_simple,,
Bosque,225,0.188714
Cultivos,442,0.090671
Herbazales,859,0.047947
Plantaciones forestales,39,1.000000


In [28]:
cobxcentroids['accepted'] = 0

In [ ]:
thidx = cobxcentroids[cobxcentroids.Cobertura_simple == 'Bosque'].index
thidx = thidx[thidx % 5 == 0]
cobxcentroids.loc[thidx, 'accepted'] = 1

thidx = cobxcentroids[cobxcentroids.Cobertura_simple == 'Cultivos'].index
thidx = thidx[thidx % 10 == 0]
cobxcentroids.loc[thidx, 'accepted'] = 1

thidx = cobxcentroids[cobxcentroids.Cobertura_simple == 'Herbazales'].index
thidx = thidx[thidx % 20 == 0]
cobxcentroids.loc[thidx, 'accepted'] = 1

cobxcentroids.loc[cobxcentroids.Cobertura_simple == 'Plantaciones forestales', 
	'accepted'] = 1

In [30]:
cobxcentroids[cobxcentroids.accepted == 1].groupby('Cobertura_simple').size()

Cobertura_simple
Bosque                     46
Cultivos                   42
Herbazales                 45
Plantaciones forestales    39
dtype: int64

In [53]:
fig, ax = plt.subplots(figsize=(8, 8))

# Set map limits
xlim = ([-74.6, -73.5])
ylim = ([3.7, 4.9])
ax.set_xlim(xlim)
ax.set_ylim(ylim)

f0 = cob[cob.Cobertura_simple.notna()
	].plot(
		ax=ax,
		column='Cobertura_simple',
		legend=True,
		cmap='rainbow'
	)

f1 = cobxcentroids[cobxcentroids.accepted == 1
	].plot(
	ax=ax,
	color='black',
	markersize=7,
	)

#ax.legend(loc='lower right')

plt.xlabel('Longitud', labelpad=15)
plt.ylabel('Latitud', labelpad=15)
plt.title('Puntos de muestreo', pad=25)
plt.tight_layout()
#plt.show()
plt.savefig('Puntos_muestreo.png', dpi=400, bbox_inches='tight')
